In [16]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
data = pd.read_csv("/content/train.csv")
data.shape




(120417, 4)

In [17]:
data = data[:100000]
data.head(5)

,title,text,tags,target
0,"ВЕРНИДУБ: «Наносимо 25 ударів, 15 у ворота, а ...",Головний тренер солігорського «Шахтаря» Юрій В...,Футбол|Інші новини|Шахтар Солігорськ|Юрій Верн...,спорт
1,"У ""Київстар"" заявляють, що їх обшукала ДФС",Про це на своїй сторінці у Facebook написав пр...,|Київстар|Новини|податкова|ДФС|обшук|,новини
2,В 2016 році 1% найзаможніших людей вперше ста...,Про це повідомляється в доповіді некомерційної...,|багатство|найбагатші люди світу|Новини|бідність|,новини
3,Шакіл О’Ніл продав шикарний маєток у Флориді в...,Легенда НБА Шакіл О’Ніл продав свій маєток у ...,США|Баскетбол|Флорида|Фото,спорт
4,"У «заблокованій» СБУ B2B Jewelry заявили, що п...",Засновник фінансової піраміди B2B Jewelry Мик...,Служба безпеки України (СБУ)|Финансовая пирами...,бізнес


In [18]:
print(data['tags'].unique(), len(data['tags'].unique()))

['Футбол|Інші новини|Шахтар Солігорськ|Юрій Вернидуб|чемпіонат Білорусі з футболу|Слуцьк'
 '|Київстар|Новини|податкова|ДФС|обшук|'
 '|багатство|найбагатші люди світу|Новини|бідність|' ...
 '|армія|Новини|Грузія|призов|'
 '|карантин|Єврокомісія|президент Єврокомісії|Урсула фон дер Ляєн|COVID-19|Новини|смерть|'
 '|ТКГ|війна на Донбасі|війна з Росією|Новини|Нормандський формат|'] 87431


In [19]:
print(data['target'].unique(), len(data['target'].unique()))


['спорт' 'новини' 'бізнес' 'політика' 'технології'] 5


**ВИБІРКА ДАНИХ**

In [20]:
targets = ['спорт', 'новини', 'бізнес']
news_in_cat_count = 2000

In [21]:
df_res = pd.DataFrame()

for target in tqdm(targets):
    df_target = data[data['target'] == target][:news_in_cat_count]
    df_res = df_res.append(df_target, ignore_index=True)

  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
df_res.shape

(6000, 4)

**ПЕРЕДОБРОБКА**

In [28]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

stopwords_ua = pd.read_csv("https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/master/stopwords_ua.txt", header=None, names=['stopwords'])
stopwords_ua.at[1983,"stopwords"] = "…"
stopwords_ua.at[1984,"stopwords"] = "«"
stopwords_ua.at[1985,"stopwords"] = "»"
stopwords_ua.at[1986,"stopwords"] = "..."
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in stopwords_ua and token != " "]
    text = " ".join(tokens)
    return text


In [30]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))

In [31]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['text'])]

  0%|          | 0/6000 [00:00<?, ?it/s]

In [32]:
len(prep_text)
prep_text[0]

'головний тренер солігорського «шахтаря» юрій вернидуб на післяматчевій прес конференції пояснив чому його підопічні програли команді «слуцьк» «такого початку я не пам ятаю у своєму житті але що є то є виправдовуватися я не буду якщо команда в тій грі – зі «смолевичами» – завдає удару зараз – або ударів з них у ворота і ми забити не можемо не знаю значить не вміємо ми забивати якщо пенальті ми не можемо реалізувати забивай і було б хвилин часу все могло б бути по іншому але що маємо те маємо у заявці не було сачівка він травму отримав ще в грі зі «смолевичами» у нього міжреберна невралгія не міг вийти у шіндагорідзе проблема з хрестоподібною зв язкою коли це вирішиться я навіть не знаю треба ще робити повторне мрт а так як я так зрозумів у нього вже була така проблема то треба їхати туди де робили але зараз все закрито – проблема чого не вистачило в обороні у першому голі просто йшла подача – і все до цього були удари не було ми навпаки нагнітали були моменти перекладина – не доводили 

In [33]:
df_res['text_prep'] = prep_text

In [34]:
df_res.head(1)

,title,text,tags,target,text_prep
0,"ВЕРНИДУБ: «Наносимо 25 ударів, 15 у ворота, а ...",Головний тренер солігорського «Шахтаря» Юрій В...,Футбол|Інші новини|Шахтар Солігорськ|Юрій Верн...,спорт,головний тренер солігорського «шахтаря» юрій в...


**Стемінг**

In [35]:
import re
from uk_stemmer import UkStemmer
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize
stemmer = UkStemmer()
stopwords_ua = pd.read_csv("https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/master/stopwords_ua.txt", header=None, names=['stopwords'])
stopwords_ua.at[1983,"stopwords"] = "…"
stopwords_ua.at[1984,"stopwords"] = "«"
stopwords_ua.at[1985,"stopwords"] = "»"
stopwords_ua.at[1986,"stopwords"] = "..."


In [ ]:
text = df_res['text_prep'][0]
word_tokenize(text) 

In [39]:
stemmed_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem_word(token) for token in tokens if token not in stopwords_ua]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_res['text_stem'] = stemmed_texts_list


  0%|          | 0/6000 [00:00<?, ?it/s]

In [40]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in stopwords_ua and token != ' ']
    return " ".join(tokens)


In [41]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in stopwords_ua and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_res['text_sw'] = sw_texts_list

  0%|          | 0/6000 [00:00<?, ?it/s]

In [42]:
df_res['text_sw'][0]

'головний тренер солігорського « шахтаря » юрій вернидуб на післяматчевій прес конференції пояснив чому його підопічні програли команді « слуцьк » « такого початку я не пам ятаю у своєму житті але що є то є виправдовуватися я не буду якщо команда в тій грі – зі « смолевичами » – завдає удару зараз – або ударів з них у ворота і ми забити не можемо не знаю значить не вміємо ми забивати якщо пенальті ми не можемо реалізувати забивай і було б хвилин часу все могло б бути по іншому але що маємо те маємо у заявці не було сачівка він травму отримав ще в грі зі « смолевичами » у нього міжреберна невралгія не міг вийти у шіндагорідзе проблема з хрестоподібною зв язкою коли це вирішиться я навіть не знаю треба ще робити повторне мрт а так як я так зрозумів у нього вже була така проблема то треба їхати туди де робили але зараз все закрито – проблема чого не вистачило в обороні у першому голі просто йшла подача – і все до цього були удари не було ми навпаки нагнітали були моменти перекладина – не 

In [43]:
df_res['text_stem'][0]

'головн тренер солігорськ « шахтар » юр вернидуб на післяматчев прес конференці поясн чом йог підопічн прогр команд « слуцьк » « так початк я не пам ят у своєм житт ал що є то є виправдовув я не буд якщ команд в ті грі – зі « смолевич » – завд удар зараз – аб удар з них у ворот і ми забит не можем не зна значит не вмієм ми забив якщ пенальт ми не можем реалізув забива і бул б хвилин час все могл б бут по інш ал що маєм те маєм у заявц не бул сачівк він травм отрим ще в грі зі « смолевич » у ньог міжреберн невралг не міг вийт у шіндагорідз проблем з хрестоподібн зв язк кол це вирішит я навіт не зна треб ще робит повторн мрт а так як я так зрозум у ньог вже бул так проблем то треб їх туд де робил ал зараз все закрит – проблем чог не вистачил в оборон у перш гол прост йшла подач – і все до цьог бул удар не бул ми навпак нагніт бул момент перекладин – не доводил до логічн завершенн у них же перш вилазк і штрафн це до воріт – метр йшла подач ніхт не торкнув і гол що говорит сумн »'

**NAIVE BAYES CLASSIFIER**


In [44]:
X = df_res['text_sw']
y = df_res['target']

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [47]:
my_tags = df_res['target'].unique()
my_tags

array(['спорт', 'новини', 'бізнес'], dtype=object)

In [48]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [50]:
%%time
nb.fit(X_train, y_train)

CPU times: user 2.44 s, sys: 153 ms, total: 2.59 s
Wall time: 2.7 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [51]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: user 349 ms, sys: 24 ms, total: 373 ms
Wall time: 376 ms


In [52]:
y_pred[0]

'спорт'

In [53]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))


accuracy 0.9011111111111111
              precision    recall  f1-score   support

       спорт       0.78      0.99      0.87       575
      новини       0.98      0.73      0.84       591
      бізнес       0.99      0.97      0.98       634

    accuracy                           0.90      1800
   macro avg       0.92      0.90      0.90      1800
weighted avg       0.92      0.90      0.90      1800



**LINEAR SUPPORT VECTOR MACHINE**


In [54]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [55]:
%%time
sgd.fit(X_train, y_train)

CPU times: user 1.33 s, sys: 139 ms, total: 1.47 s
Wall time: 1.42 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [56]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 340 ms, sys: 7.66 ms, total: 348 ms
Wall time: 351 ms


In [57]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9583333333333334
              precision    recall  f1-score   support

       спорт       0.92      0.98      0.94       575
      новини       0.97      0.91      0.94       591
      бізнес       0.99      0.99      0.99       634

    accuracy                           0.96      1800
   macro avg       0.96      0.96      0.96      1800
weighted avg       0.96      0.96      0.96      1800



**LOGISTIC REGRESSION**

In [58]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [59]:
%%time
logreg.fit(X_train, y_train)

CPU times: user 9.08 s, sys: 6.84 s, total: 15.9 s
Wall time: 9.17 s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [60]:
%%time
y_pred = logreg.predict(X_test)


CPU times: user 360 ms, sys: 3.68 ms, total: 363 ms
Wall time: 365 ms


In [61]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9633333333333334
              precision    recall  f1-score   support

       спорт       0.94      0.96      0.95       575
      новини       0.95      0.94      0.95       591
      бізнес       0.99      0.99      0.99       634

    accuracy                           0.96      1800
   macro avg       0.96      0.96      0.96      1800
weighted avg       0.96      0.96      0.96      1800



**ПЕРЕВІРКА НА ПРИКЛАДІ**

In [62]:
business_text = "Долар США вважається символом надійності, безпеки та економічного процвітання. Він займає незаперечне домінуюче становище у середині ХХ століття. враження непереможного титану. Однак ера панування долара як основної світової резервної валюти повільно добігає кінця. Найбільші банки пророкують йому різкий спад уже наступного року, а відомий економіст Стівен Роуч упевнений, що американська валюта може знецінитись на третину. Причинами обвалення стануть скорочення заощаджень населення, зростання державного боргу США та посилення Китаю. Захід доларового диктату – у матеріалі «Ленты.ру». Надмірні привілеї Успіх американської економіки у XX столітті був багато в чому обумовлений домінуючою роллю долара. У свою чергу досягнення цієї ролі стало результатом політичної та військової переваги, яку США набули після Першої світової війни. Досі позиція долара у світі фінансів є головною основою процвітання США. Однак у 2002 році намітилася довгострокова тенденція до послаблення американської валюти, яка спостерігається і сьогодні. Справді, останнім часом долар почуватиметься не дуже добре. Зокрема, липень виявився дуже складним місяцем для американської валюти, яка відновила."

In [64]:
business_text = remove_multiple_spaces(remove_numbers(remove_punctuation(business_text.lower())))
business_text = remove_stop_words(business_text)

In [67]:
business_text

'долар сша вважається символом надійності безпеки та економічного процвітання він займає незаперечне домінуюче становище у середині хх століття враження непереможного титану однак ера панування долара як основної світової резервної валюти повільно добігає кінця найбільші банки пророкують йому різкий спад уже наступного року а відомий економіст стівен роуч упевнений що американська валюта може знецінитись на третину причинами обвалення стануть скорочення заощаджень населення зростання державного боргу сша та посилення китаю захід доларового диктату – у матеріалі « ленты ру » надмірні привілеї успіх американської економіки у xx столітті був багато в чому обумовлений домінуючою роллю долара у свою чергу досягнення цієї ролі стало результатом політичної та військової переваги яку сша набули після першої світової війни досі позиція долара у світі фінансів є головною основою процвітання сша однак у році намітилася довгострокова тенденція до послаблення американської валюти яка спостерігаєтьс

In [68]:
business_pred = logreg.predict([business_text])
business_pred

array(['бізнес'], dtype=object)

In [69]:
sport_text = "І в чому тоді був не правий Флік, коли вже за кілька хвилин зробив аналогічну заміну з появою в грі Нікласа Фюллькруга, якого за виступами за бременський «Вердер» знають хіба що затяті фанаті німецької Бундесліги? А він тим часом з’явився на полі, і одразу ж став гідною мішенню для передач партнерів. Одна з таких, на 83 хвилині від Мусіали, який перевів пас із глибини від Сане до правого флангу штрафного, завершилась ударом у дальній кут. Завдяки цьому «Бундестім» урятували не тільки цю гру, але й власну можливість впливати на долю другої перепустки до плей-оф."

In [70]:
sport_text = remove_multiple_spaces(remove_numbers(remove_punctuation(sport_text.lower())))
sport_text = remove_stop_words(sport_text)

In [71]:
sport_pred = logreg.predict([sport_text])
sport_pred

array(['спорт'], dtype=object)

In [72]:
news_text = "На початку тижня Україна перейде до планових графіків відключення електрики — YASNOОрієнтовно 28-29 листопада в Україні буде поновлено графік планових відключень електроенергії. Про це в ефірі загальнонаціонального телемарафону заявив генеральний директор енергетичної компанії YASNO Сергій Коваленко.Говоритимемо моїми персональними прогнозами. Я розраховую, що завтра чи у вівторок (28 чи 29 листопада — ред.) ми повернемось до звичних графіків. Але це залежить від запуску генерації та деяких моментів, — зазначив він."

In [73]:
news_text = remove_multiple_spaces(remove_numbers(remove_punctuation(news_text.lower())))
news_text = remove_stop_words(news_text)

In [74]:
news_pred = logreg.predict([news_text])
news_pred

array(['новини'], dtype=object)